In [1]:
import gym
import numpy as np
import copy, random, os, subprocess, cv2
import tensorflow as tf
import keras.backend as K
from tensorflow.keras import layers, models, regularizers

/opt/conda/lib/python3.7/site-packages/flatbuffers/compat.py:19: DeprecationWarning: the imp module is deprecated in favour of importlib; see the module's documentation for alternative uses
  import imp
/opt/conda/lib/python3.7/site-packages/keras_preprocessing/image/utils.py:23: DeprecationWarning: NEAREST is deprecated and will be removed in Pillow 10 (2023-07-01). Use Resampling.NEAREST or Dither.NONE instead.
  'nearest': pil_image.NEAREST,
/opt/conda/lib/python3.7/site-packages/keras_preprocessing/image/utils.py:24: DeprecationWarning: BILINEAR is deprecated and will be removed in Pillow 10 (2023-07-01). Use Resampling.BILINEAR instead.
  'bilinear': pil_image.BILINEAR,
/opt/conda/lib/python3.7/site-packages/keras_preprocessing/image/utils.py:25: DeprecationWarning: BICUBIC is deprecated and will be removed in Pillow 10 (2023-07-01). Use Resampling.BICUBIC instead.
  'bicubic': pil_image.BICUBIC,
/opt/conda/lib/python3.7/site-packages/keras_preprocessing/image/utils.py:28: Depreca

In [2]:
MODEL_NAME = 'model06'
BUCKET = 'gs://etsuji-car-racing-v2'
os.environ['BUCKET'] = BUCKET
!gsutil mb -c regional -l us-west1 $BUCKET
!gsutil ls $BUCKET

Creating gs://etsuji-car-racing-v2/...
ServiceException: 409 A Cloud Storage bucket named 'etsuji-car-racing-v2' already exists. Try another name. Bucket names must be globally unique across all Google Cloud projects, including those outside of your organization.
gs://etsuji-car-racing-v2/model04/
gs://etsuji-car-racing-v2/model05/
gs://etsuji-car-racing-v2/model06/
gs://etsuji-car-racing-v2/model07/


In [3]:
class ApplyL1Weight(layers.Layer):
    def __init__(self, l1=0.01, **kwargs):
        self.filter_shape = None
        self.l1 = l1
        super(ApplyL1Weight, self).__init__(**kwargs)
        
    def build(self, input_shape):
        self.kernel = self.add_weight(name='weights', shape=[input_shape[3]],
                                      regularizer=regularizers.L1(self.l1))
        self.filter_shape = input_shape

    def get_config(self):
        config = super().get_config()
        config.update({
            'l1': self.l1,
        })
        return config
    
    def call(self, inputs, **kwargs):
        return inputs * kernel
    
# Base model
class QValue:
    def __init__(self):
        self.model = self.build_model()

    def build_model(self):
        cnn_input = layers.Input(shape=(48, 48, 3), name='cnn_input')
        cnn1 = layers.Conv2D(16, (5, 5), padding='same',
                             use_bias=True, activation='relu',
                             name='cnn1')(cnn_input)
        pool1 = layers.MaxPooling2D((2, 2), name='pool1')(cnn1)
        cnn2 = layers.Conv2D(32, (5, 5), padding='same',
                             use_bias=True, activation='relu',
                             name='cnn2')(pool1)        
        pool2 = layers.MaxPooling2D((2, 2), name='pool2')(cnn2)
        weighted_filters = ApplyL1Weight(name='weighted_filters')(pool2)

        cnn_flatten = layers.Flatten(name='flatten')(weighted_filters)
        action_input = layers.Input(shape=(5,), name='action_input')
        combined = layers.concatenate([cnn_flatten, action_input], name='concat')
        hidden1 = layers.Dense(1024, activation='relu', name='dense1')(combined)
        hidden2 = layers.Dense(512, activation='relu', name='dense2')(hidden1)
        q_value = layers.Dense(1, name='output')(hidden2)

        model = models.Model(inputs=[cnn_input, action_input], outputs=q_value)
        model.compile(loss='mse')
        return model

    def get_action(self, state):
        states = []
        actions = []
        for a in range(5):
            states.append(np.array(state))
            action_onehot = np.zeros(5)
            action_onehot[a] = 1
            actions.append(action_onehot)
  
        q_values = self.model.predict([np.array(states), np.array(actions)])
        optimal_action = np.argmax(q_values)
        return optimal_action, q_values[optimal_action][0]

In [4]:
def join_frames(o0, o1, o2):
    gray_image0 = cv2.cvtColor(cv2.resize(o0, (48, 48)), cv2.COLOR_RGB2GRAY)
    gray_image1 = cv2.cvtColor(cv2.resize(o1, (48, 48)), cv2.COLOR_RGB2GRAY)
    gray_image2 = cv2.cvtColor(cv2.resize(o2, (48, 48)), cv2.COLOR_RGB2GRAY)
    
    return np.array(
        [gray_image0.transpose(),
         gray_image1.transpose(),
         gray_image2.transpose()]).transpose()

In [5]:
def get_episode(environ, q_value, epsilon):
    episode = []
    o0 = environ.reset()
    o1 = copy.deepcopy(o0)
    o2 = copy.deepcopy(o0)
    total_r = 0

    if epsilon > 0:
        keep_count = 3
    else:
        keep_count = 1

    c = 0
    while True:
        if c % keep_count == 0: # Get new action
            if np.random.random() < epsilon:
                a = np.random.randint(5)
            else:
                a, _ = q_value.get_action(join_frames(o0, o1, o2))
        c += 1
        o_new, r, done, inf = environ.step(a)                
        total_r += r

        # Terminate episode when total reward becomes negative
        if total_r < 0:
            done = 1

        if done:
            # Terminal state is to achive more than 990 or get out of the field.
            if total_r > 990 or r < -99:
                episode.append((join_frames(o0, o1, o2), a, r, None))
            break
        else:
            episode.append((join_frames(o0, o1, o2), a, r, join_frames(o1, o2, o_new)))
        o0, o1, o2 = o1, o2, o_new

    print('epsilon={}, episode length={}, total rewards={}'.format(epsilon, len(episode), total_r))
    return episode, total_r

In [6]:
def train(environ, q_value, epsilon, checkpoint=0):
    if checkpoint > 0:
        filename = 'car-racing-v2-{}-{}.hd5'.format(checkpoint, MODEL_NAME)
        subprocess.run(['gsutil', 'cp', '{}/{}/{}'.format(BUCKET, MODEL_NAME, filename), './'])
        print('load model {}'.format(filename))
        q_value.model = models.load_model(filename)
        os.remove(filename)

    experience = []
    good_experience = []
    best_r = [-100, -100, -100]

    for n in range(checkpoint + 1, checkpoint + 1000):
        print('iteration {}'.format(n))

        total_len = 0
        if n % 3 == 0:
            print('Testing the current performance...')
            episode, total_r = get_episode(environ, q_value, epsilon=0)
            with open('result.txt', 'a') as f:
                f.write('{},{},{},{}\n'.format(n, epsilon, len(episode), total_r))
            filename = 'car-racing-v2-{}-{}.hd5'.format(n, MODEL_NAME)
            q_value.model.save(filename, save_format='h5')
            subprocess.run(['gsutil', '-m', 'cp',
                            '{}'.format(filename), '{}/{}/'.format(BUCKET, MODEL_NAME)])
            os.remove(filename)
            experience += episode
            total_len += len(episode)

        while total_len < 500:
            episode, total_r = get_episode(environ, q_value, epsilon)
            total_len += len(episode)
            experience += episode

            # Keep the top 3 episodes
            if total_r > min(best_r):
                best_r = best_r[1:] + [total_r]
                good_experience += episode
                if len(good_experience) > 999 * 3:
                    good_experience = good_experience[-999 * 3:]

            
        if len(experience) > 999 * 5: # remember last 5 episodes
            experience = experience[-999 * 5:]

        epsilon = (epsilon - 0.2) * 0.99 + 0.2

        print('Training the model...')
        # Use latest episode + past episodes (sampling) + top 3 episode (sampling)
        latest_experience = experience[-total_len:]
        past_experience = experience[:-total_len]
        examples = latest_experience + \
            random.sample(past_experience, min(len(past_experience), 999)) + \
            random.sample(good_experience, min(len(good_experience), 999))
        
        # Show some statistics
        print('experience length={}'.format(len(experience)))
        print('number of examples={}'.format(len(examples)))
        print('best total reward = ', best_r)
        np.random.shuffle(examples)
                        
        states, actions, labels = [], [], []
        for state, a, r, state_new in examples:
            states.append(np.array(state))

            action_onehot = np.zeros(5)
            action_onehot[a] = 1
            actions.append(action_onehot)
            
            if state_new is None:   # Terminal state
                q_new = 0
            else:
                _, q_new = q_value.get_action(state_new)
            labels.append(np.array(r + q_new))

        hist = q_value.model.fit(
            [np.array(states), np.array(actions)], np.array(labels),
            batch_size=50, epochs=10, verbose=0)
        print('loss = {}'.format(hist.history['loss']))

In [7]:
env = gym.make("CarRacing-v2", continuous=False)
q_value = QValue()
q_value.model.summary()

/opt/conda/lib/python3.7/site-packages/gym/core.py:330: DeprecationWarning: WARN: Initializing wrapper in old step API which returns one bool instead of two. It is recommended to set `new_step_api=True` to use new step API. This will be the default behaviour in future.
  "Initializing wrapper in old step API which returns one bool instead of two. It is recommended to set `new_step_api=True` to use new step API. This will be the default behaviour in future."
/opt/conda/lib/python3.7/site-packages/gym/wrappers/step_api_compatibility.py:40: DeprecationWarning: WARN: Initializing environment in old step API which returns one bool instead of two. It is recommended to set `new_step_api=True` to use new step API. This will be the default behaviour in future.
  "Initializing environment in old step API which returns one bool instead of two. It is recommended to set `new_step_api=True` to use new step API. This will be the default behaviour in future."
2022-08-01 23:46:48.868330: I tensorflow/s

Model: "model"
__________________________________________________________________________________________________
 Layer (type)                   Output Shape         Param #     Connected to                     
 cnn_input (InputLayer)         [(None, 48, 48, 3)]  0           []                               
                                                                                                  
 cnn1 (Conv2D)                  (None, 48, 48, 16)   1216        ['cnn_input[0][0]']              
                                                                                                  
 pool1 (MaxPooling2D)           (None, 24, 24, 16)   0           ['cnn1[0][0]']                   
                                                                                                  
 cnn2 (Conv2D)                  (None, 24, 24, 32)   12832       ['pool1[0][0]']                  
                                                                                              

2022-08-01 23:46:48.883051: I tensorflow/core/platform/cpu_feature_guard.cc:151] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.
2022-08-01 23:46:48.884062: I tensorflow/stream_executor/cuda/cuda_gpu_executor.cc:936] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero
2022-08-01 23:46:48.884753: I tensorflow/stream_executor/cuda/cuda_gpu_executor.cc:936] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero
2022-08-01 23:46:48.885356: I tensorflow/stream_executor/cuda/cuda_gpu_executor.cc:936] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zer

In [ ]:
train(env, q_value, epsilon=1.0, checkpoint=0)

iteration 1
epsilon=1.0, episode length=790, total rewards=-0.04861660079024974
Training the model...
experience length=790
number of examples=1580
best total reward =  [-100, -100, -0.04861660079024974]


2022-08-01 23:46:57.596929: I tensorflow/stream_executor/cuda/cuda_dnn.cc:368] Loaded cuDNN version 8200


loss = [30063.845703125, 18.222278594970703, 5.5592942237854, 1.5709571838378906, 0.6793814897537231, 0.6274317502975464, 0.6696242094039917, 0.7548159956932068, 0.5571935772895813, 0.6653337478637695]
iteration 2
epsilon=0.992, episode length=131, total rewards=-0.04210526315789062
epsilon=0.992, episode length=111, total rewards=-0.005970149253715834
epsilon=0.992, episode length=182, total rewards=-0.051824817518234534
epsilon=0.992, episode length=185, total rewards=-0.08148148148143197
Training the model...
experience length=1399
number of examples=2398
best total reward =  [-0.04861660079024974, -0.04210526315789062, -0.005970149253715834]
loss = [0.4442785680294037, 0.44378870725631714, 0.3842591643333435, 0.371902734041214, 0.3642255961894989, 0.3419865667819977, 0.34576311707496643, 0.32436221837997437, 0.3210403025150299, 0.31514447927474976]
iteration 3
Testing the current performance...
epsilon=0, episode length=999, total rewards=60.00000000000463


Copying file://car-racing-v2-3-model06.hd5 [Content-Type=application/octet-stream]...
/ [1/1 files][ 40.2 MiB/ 40.2 MiB] 100% Done                                    
Operation completed over 1 objects/40.2 MiB.                                     


Training the model...
experience length=2398
number of examples=2997
best total reward =  [-0.04861660079024974, -0.04210526315789062, -0.005970149253715834]
loss = [0.46547800302505493, 0.39634308218955994, 0.3981923460960388, 0.3745150566101074, 0.35997647047042847, 0.38221824169158936, 0.345010906457901, 0.36476361751556396, 0.3412971794605255, 0.34975534677505493]
iteration 4
epsilon=0.9762392000000002, episode length=69, total rewards=-0.006993006992998846
epsilon=0.9762392000000002, episode length=95, total rewards=-0.015335463258780296
epsilon=0.9762392000000002, episode length=168, total rewards=-0.06498316498312318
epsilon=0.9762392000000002, episode length=198, total rewards=-0.0324503311257984
Training the model...
experience length=2928
number of examples=2528
best total reward =  [-0.005970149253715834, -0.006993006992998846, -0.015335463258780296]
loss = [0.3834962546825409, 0.35106992721557617, 0.2941764295101166, 0.30434924364089966, 0.28610384464263916, 0.2967178523540

...

Training the model...
experience length=4995
number of examples=2997
best total reward =  [142.95774647887595, 122.22222222222359, 146.79487179487492]
loss = [1.480142593383789, 1.779833436012268, 1.2497731447219849, 1.231669545173645, 1.129136085510254, 1.1506279706954956, 1.0266485214233398, 0.9948644042015076, 0.9564331769943237, 0.991378903388977]
iteration 163
epsilon=0.3570332112184209, episode length=72, total rewards=-0.05362318840578806
epsilon=0.3570332112184209, episode length=314, total rewards=-0.03146853146843506
epsilon=0.3570332112184209, episode length=227, total rewards=-0.07272727272720769
Training the model...
experience length=4995
number of examples=2611
best total reward =  [142.95774647887595, 122.22222222222359, 146.79487179487492]
loss = [1.3468292951583862, 1.1730495691299438, 1.1583112478256226, 1.0347445011138916, 1.050458312034607, 1.0981481075286865, 0.8614773750305176, 0.9322991967201233, 0.8169112801551819, 0.8394115567207336]
iteration 164
epsilon=0.35

Copying file://car-racing-v2-165-model06.hd5 [Content-Type=application/octet-stream]...
/ [1/1 files][ 40.2 MiB/ 40.2 MiB] 100% Done                                    
Operation completed over 1 objects/40.2 MiB.                                     


Training the model...
experience length=4995
number of examples=2997
best total reward =  [142.95774647887595, 122.22222222222359, 146.79487179487492]
loss = [1.552110195159912, 1.4846656322479248, 1.3074591159820557, 1.2654331922531128, 1.2642605304718018, 1.1741490364074707, 1.1423851251602173, 1.2645419836044312, 1.0964637994766235, 1.1280581951141357]
iteration 166
epsilon=0.35236916781202265, episode length=999, total rewards=37.54646840148572
Training the model...
experience length=4995
number of examples=2997
best total reward =  [142.95774647887595, 122.22222222222359, 146.79487179487492]
loss = [1.541200876235962, 1.2677712440490723, 1.1061948537826538, 1.1015833616256714, 1.0619280338287354, 0.9422764778137207, 0.9437036514282227, 0.9532618522644043, 0.9840382933616638, 0.8801000118255615]
iteration 167
epsilon=0.35084547613390243, episode length=919, total rewards=-0.04597701149501285
Training the model...
experience length=4995
number of examples=2917
best total reward =  [

Copying file://car-racing-v2-168-model06.hd5 [Content-Type=application/octet-stream]...
/ [1/1 files][ 40.2 MiB/ 40.2 MiB] 100% Done                                    
Operation completed over 1 objects/40.2 MiB.                                     


Training the model...
experience length=4995
number of examples=2997
best total reward =  [142.95774647887595, 122.22222222222359, 146.79487179487492]
